In [ ]:
import os
HOME = os.getcwd()
HOME = '/content'
print(HOME)

/content


## Resizing IR videos

In [ ]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cv2
import os

input_directory = "/content/gdrive/MyDrive/FYP_Drone/ObjectTracking/Video_IR"
output_directory = "/content/gdrive/MyDrive/FYP_Drone/ObjectTracking/Resized_Video_IR/"

os.makedirs(output_directory, exist_ok=True)

fourcc = cv2.VideoWriter_fourcc(*"XVID")

# List all video files in the input directory
video_files = [f for f in os.listdir(input_directory) if f.endswith('.mp4')]

for video_file in video_files:
    input_path = os.path.join(input_directory, video_file)
    output_path = os.path.join(output_directory, video_file)

    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) * 2
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) * 2

    writer = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    success, img = cap.read()
    while success:
        writer.write(cv2.resize(img, (w, h)))
        success, img = cap.read()

    cap.release()
    writer.release()

print("Resizing completed.")

Resizing completed.


## Generate center coordinates of bounding boxes

In [ ]:
!ls /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/Video_V

V_DRONE_001.mp4  V_DRONE_016.mp4  V_DRONE_044.mp4  V_DRONE_066.mp4  V_DRONE_077.mp4
V_DRONE_002.mp4  V_DRONE_017.mp4  V_DRONE_054.mp4  V_DRONE_067.mp4  V_DRONE_078.mp4
V_DRONE_003.mp4  V_DRONE_018.mp4  V_DRONE_055.mp4  V_DRONE_068.mp4  V_DRONE_080.mp4
V_DRONE_004.mp4  V_DRONE_021.mp4  V_DRONE_058.mp4  V_DRONE_070.mp4  V_DRONE_081.mp4
V_DRONE_006.mp4  V_DRONE_022.mp4  V_DRONE_059.mp4  V_DRONE_071.mp4  V_DRONE_082.mp4
V_DRONE_007.mp4  V_DRONE_023.mp4  V_DRONE_060.mp4  V_DRONE_072.mp4  V_DRONE_086.mp4
V_DRONE_012.mp4  V_DRONE_024.mp4  V_DRONE_062.mp4  V_DRONE_073.mp4  V_DRONE_087.mp4
V_DRONE_013.mp4  V_DRONE_025.mp4  V_DRONE_063.mp4  V_DRONE_074.mp4  V_DRONE_091.mp4
V_DRONE_014.mp4  V_DRONE_040.mp4  V_DRONE_064.mp4  V_DRONE_075.mp4  V_DRONE_092.mp4
V_DRONE_015.mp4  V_DRONE_042.mp4  V_DRONE_065.mp4  V_DRONE_076.mp4  V_DRONE_096.mp4


In [ ]:
!ls /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/Video_IR

IR_DRONE_001.mp4  IR_DRONE_012.mp4  IR_DRONE_028.mp4  IR_DRONE_039.mp4	IR_DRONE_074.mp4
IR_DRONE_002.mp4  IR_DRONE_013.mp4  IR_DRONE_029.mp4  IR_DRONE_040.mp4	IR_DRONE_095.mp4
IR_DRONE_003.mp4  IR_DRONE_016.mp4  IR_DRONE_030.mp4  IR_DRONE_041.mp4	IR_DRONE_100.mp4
IR_DRONE_004.mp4  IR_DRONE_017.mp4  IR_DRONE_031.mp4  IR_DRONE_042.mp4	IR_DRONE_101.mp4
IR_DRONE_005.mp4  IR_DRONE_018.mp4  IR_DRONE_032.mp4  IR_DRONE_044.mp4	IR_DRONE_109.mp4
IR_DRONE_006.mp4  IR_DRONE_023.mp4  IR_DRONE_033.mp4  IR_DRONE_045.mp4	IR_DRONE_119.mp4
IR_DRONE_008.mp4  IR_DRONE_024.mp4  IR_DRONE_034.mp4  IR_DRONE_046.mp4	IR_DRONE_120.mp4
IR_DRONE_009.mp4  IR_DRONE_025.mp4  IR_DRONE_035.mp4  IR_DRONE_048.mp4	IR_DRONE_128.mp4
IR_DRONE_010.mp4  IR_DRONE_026.mp4  IR_DRONE_036.mp4  IR_DRONE_049.mp4	IR_DRONE_129.mp4
IR_DRONE_011.mp4  IR_DRONE_027.mp4  IR_DRONE_038.mp4  IR_DRONE_056.mp4	IR_DRONE_137.mp4


### Generate 50 frames for all 100 videos (50 visible + 50 IR)

In [ ]:
OT = '/content/gdrive/MyDrive/FYP_Drone/ObjectTracking'

In [ ]:
import cv2
import os

# Paths
video_directory = f"{OT}/Video_IR/"
frames_directory = f"{OT}/FrameIR/"

# Create frames directory if it doesn't exist
os.makedirs(frames_directory, exist_ok=True)

# Iterate through videos in the Video_V directory
for video_name in os.listdir(video_directory):
    video_path = os.path.join(video_directory, video_name)

    # Create frames directory for the current video
    video_frames_dir = os.path.join(frames_directory, video_name.split('.')[0])
    os.makedirs(video_frames_dir, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if fps == 0:
        print(f"Error: Could not read FPS for video {video_name}. Skipping this video.")
        cap.release()
        continue

    video_duration = total_frames / fps  # Duration in seconds

    # Ensure the video is 10 seconds long
    if video_duration != 10:
        print(f"Warning: Video {video_name} duration is not 10 seconds, it's {video_duration} seconds.")

    frames_to_extract = 50
    frame_interval = total_frames // frames_to_extract

    # Extract frames
    extracted_frames = 0
    for i in range(frames_to_extract):
        frame_index = i * frame_interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(video_frames_dir, f"frame_{extracted_frames:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        extracted_frames += 1

    cap.release()
    print(f"Extracted {extracted_frames} frames from {video_name}")

Extracted 50 frames from IR_DRONE_035.mp4
Extracted 50 frames from IR_DRONE_028.mp4
Extracted 50 frames from IR_DRONE_030.mp4
Extracted 50 frames from IR_DRONE_137.mp4
Extracted 50 frames from IR_DRONE_006.mp4
Extracted 50 frames from IR_DRONE_026.mp4
Extracted 50 frames from IR_DRONE_011.mp4
Extracted 50 frames from IR_DRONE_002.mp4
Extracted 50 frames from IR_DRONE_003.mp4
Extracted 50 frames from IR_DRONE_036.mp4
Extracted 50 frames from IR_DRONE_023.mp4
Extracted 50 frames from IR_DRONE_040.mp4
Extracted 50 frames from IR_DRONE_013.mp4
Extracted 50 frames from IR_DRONE_012.mp4
Extracted 50 frames from IR_DRONE_034.mp4
Extracted 50 frames from IR_DRONE_049.mp4
Extracted 50 frames from IR_DRONE_041.mp4
Extracted 50 frames from IR_DRONE_010.mp4
Extracted 50 frames from IR_DRONE_027.mp4
Extracted 50 frames from IR_DRONE_038.mp4
Extracted 50 frames from IR_DRONE_048.mp4
Extracted 50 frames from IR_DRONE_042.mp4
Extracted 50 frames from IR_DRONE_018.mp4
Extracted 50 frames from IR_DRONE_

### Get labels for each frame


In [ ]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.9/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
HOME = '/content/gdrive/MyDrive/FYP_Drone/ObjectDetection/V_IR/Result_Metrics/V_YOLOv8s'

In [ ]:
import os
import shutil

# Define paths
frame_directory = f"{OT}/FrameV"
results_directory = f"{OT}/LabelsV"

# Iterate through each video folder in the Frame directory
for video_folder in os.listdir(frame_directory):
    # Construct full paths
    video_folder_path = os.path.join(frame_directory, video_folder)
    result_folder_path = os.path.join(results_directory, video_folder)

    # Create a folder for the results of each video
    os.makedirs(result_folder_path, exist_ok=True)

    !yolo task=detect mode=predict model={HOME}/train/weights/best.pt conf=0.25 source={video_folder_path} save=True save_txt=True

    # Find the directory with the detection results for the current video
    detection_dirs = [d for d in os.listdir(f"/content/runs/detect") if d.startswith("predict")]
    detection_dirs.sort(key=lambda x: os.stat(os.path.join(f"/content/runs/detect", x)).st_mtime, reverse=True)  # Sort by modification time
    detection_results_dir = None
    for detect_dir in detection_dirs:
        if os.path.exists(os.path.join(f"/content/runs/detect", detect_dir, "labels")):
            detection_results_dir = os.path.join(f"/content/runs/detect", detect_dir, "labels")
            break

    # Copy the label files to the corresponding result folder
    if detection_results_dir:
        for file in os.listdir(detection_results_dir):
            shutil.copy(os.path.join(detection_results_dir, file), result_folder_path)

        print(f"Labels saved for {video_folder} in {result_folder_path}")
    else:
        print(f"No detection results found for {video_folder}")

2024-05-24 02:18:45.858348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 02:18:45.858404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 02:18:46.010242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 02:18:46.280636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 02:18:47.435149: W tensorflow/compiler/tf2

In [ ]:
HOME = '/content/gdrive/MyDrive/FYP_Drone/ObjectDetection/V_IR/Result_Metrics/IR_YOLOv8n'

In [ ]:
import os
import shutil

# Define paths
frame_directory = f"{OT}/FrameIR"
results_directory = f"{OT}/LabelsIR"

# Iterate through each video folder in the Frame directory
for video_folder in os.listdir(frame_directory):
    # Construct full paths
    video_folder_path = os.path.join(frame_directory, video_folder)
    result_folder_path = os.path.join(results_directory, video_folder)

    # Create a folder for the results of each video
    os.makedirs(result_folder_path, exist_ok=True)

    !yolo task=detect mode=predict model={HOME}/train/weights/best.pt conf=0.25 source={video_folder_path} save=True save_txt=True

    # Find the directory with the detection results for the current video
    detection_dirs = [d for d in os.listdir(f"/content/runs/detect") if d.startswith("predict")]
    detection_dirs.sort(key=lambda x: os.stat(os.path.join(f"/content/runs/detect", x)).st_mtime, reverse=True)  # Sort by modification time
    detection_results_dir = None
    for detect_dir in detection_dirs:
        if os.path.exists(os.path.join(f"/content/runs/detect", detect_dir, "labels")):
            detection_results_dir = os.path.join(f"/content/runs/detect", detect_dir, "labels")
            break

    # Copy the label files to the corresponding result folder
    if detection_results_dir:
        for file in os.listdir(detection_results_dir):
            shutil.copy(os.path.join(detection_results_dir, file), result_folder_path)

        print(f"Labels saved for {video_folder} in {result_folder_path}")
    else:
        print(f"No detection results found for {video_folder}")

2024-05-24 03:32:03.150121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 03:32:03.150212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 03:32:03.152055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 03:32:03.162891: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 03:32:04.881719: W tensorflow/compiler/tf2

### Generate CSV files

In [ ]:
import os
import glob
import pandas as pd

# Define paths
detection_results_dir = f"{OT}/LabelsV"
output_csv_dir = f"{OT}/FileV"

# Ensure the output directory exists
os.makedirs(output_csv_dir, exist_ok=True)

# Iterate through each folder in the Labels directory
for folder in os.listdir(detection_results_dir):
    folder_path = os.path.join(detection_results_dir, folder)

    # Output CSV file path
    output_csv_file = os.path.join(output_csv_dir, f"{folder}.csv")

    # List to hold aggregated data
    data = []

    # Get all the result files and sort them to maintain the order
    result_files = sorted(glob.glob(os.path.join(folder_path, "*.txt")))

    for result_file in result_files:
        frame_id = os.path.basename(result_file).split('.')[0]  # Extract frame identifier
        class_id_2_count = 0  # Counter for class_id 2

        with open(result_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            # Each line in the result file is in the format:
            # class_id x_center y_center width height (normalized to [0, 1])
            parts = line.strip().split()
            if parts[0] == '2':  # Check if class_id is 2
                class_id, x_center, y_center = parts[0], parts[1], parts[2]
                data.append([frame_id, class_id, x_center, y_center])
                class_id_2_count += 1

        # Print filename if there are two lines of class_id 2
        if class_id_2_count >= 2:
            print(f"File with multiple class_id 2 entries: {result_file}")

    # Convert the data to a DataFrame
    df = pd.DataFrame(data, columns=['frame_id', 'class_id', 'x_center', 'y_center'])

    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_file, index=False)

    print(f"Converted {len(result_files)} result files to a single CSV file: {output_csv_file}")

Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileV/V_DRONE_016.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileV/V_DRONE_014.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileV/V_DRONE_091.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileV/V_DRONE_080.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileV/V_DRONE_071.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileV/V_DRONE_067.csv
File with multiple class_id 2 entries: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/LabelsV/V_DRONE_077/frame_0008.txt
File with multiple class_id 2 entries: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/LabelsV/V_DRONE_077/frame_0020.txt
Converted 50 result files to a single CS

In [ ]:
import os
import glob
import pandas as pd

# Define paths
detection_results_dir = f"{OT}/LabelsIR"
output_csv_dir = f"{OT}/FileIR"

# Ensure the output directory exists
os.makedirs(output_csv_dir, exist_ok=True)

# Iterate through each folder in the Labels directory
for folder in os.listdir(detection_results_dir):
    folder_path = os.path.join(detection_results_dir, folder)

    # Output CSV file path
    output_csv_file = os.path.join(output_csv_dir, f"{folder}.csv")

    # List to hold aggregated data
    data = []

    # Get all the result files and sort them to maintain the order
    result_files = sorted(glob.glob(os.path.join(folder_path, "*.txt")))

    for result_file in result_files:
        frame_id = os.path.basename(result_file).split('.')[0]  # Extract frame identifier
        class_id_2_count = 0  # Counter for class_id 2

        with open(result_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            # Each line in the result file is in the format:
            # class_id x_center y_center width height (normalized to [0, 1])
            parts = line.strip().split()
            if parts[0] == '2':  # Check if class_id is 2
                class_id, x_center, y_center = parts[0], parts[1], parts[2]
                data.append([frame_id, class_id, x_center, y_center])
                class_id_2_count += 1

        # Print filename if there are two lines of class_id 2
        if class_id_2_count >= 2:
            print(f"File with multiple class_id 2 entries: {result_file}")

    # Convert the data to a DataFrame
    df = pd.DataFrame(data, columns=['frame_id', 'class_id', 'x_center', 'y_center'])

    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_file, index=False)

    print(f"Converted {len(result_files)} result files to a single CSV file: {output_csv_file}")

Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_026.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_011.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_002.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_003.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_036.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_023.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_040.csv
Converted 50 result files to a single CSV file: /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/FileIR/IR_DRONE_013.csv
Converted 50 result files to a s

### Visualise

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

# Define paths
csv_directory = f"{OT}/FileV"
output_directory = f"{OT}/GTPlotV"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Iterate through each file in the CSV directory
for file_name in os.listdir(csv_directory):
    file_path = os.path.join(csv_directory, file_name)

    try:
        # Read the file based on its extension
        if file_name.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_name.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Skipping unsupported file format: {file_name}")
            continue

        # Check if necessary columns exist
        if 'x_center' not in df.columns or 'y_center' not in df.columns:
            print(f"Skipping file {file_name} as it doesn't contain necessary columns")
            continue

        sorted_coords = list(zip(df['x_center'], df['y_center']))

        # Plot the center coordinates
        plt.figure(figsize=(10, 8))
        plt.scatter([coord[0] for coord in sorted_coords], [coord[1] for coord in sorted_coords], marker='o', color='green', label='Ground Truth')

        # Annotate the points with sequential numbers
        for i, (x, y) in enumerate(sorted_coords):
            plt.annotate(str(i + 1), (x, y), textcoords="offset points", xytext=(0, 10), ha='center', color='red')

        plt.xlabel('X coordinate')
        plt.ylabel('Y coordinate')
        plt.title('Center Coordinates of Bounding Boxes')
        plt.xlim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.ylim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.gca().invert_yaxis()  # YOLO format typically has y-axis inverted
        plt.legend()
        plt.grid(True)

        # Save the plot in the GT folder with the same name as the CSV/XLSX file but with a .png extension
        output_file = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.png")
        plt.savefig(output_file, transparent=True)
        plt.close()  # Close the plot to free memory

        print(f"Plot saved for {file_name} in {output_file}")
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

print("Plotting completed.")

Plot saved for V_DRONE_016.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_016.png
Plot saved for V_DRONE_014.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_014.png
Plot saved for V_DRONE_091.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_091.png
Plot saved for V_DRONE_080.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_080.png
Plot saved for V_DRONE_071.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_071.png
Plot saved for V_DRONE_067.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_067.png
Plot saved for V_DRONE_055.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_055.png
Plot saved for V_DRONE_059.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_059.png
Plot saved for V_DRONE_004.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV/V_DRONE_004.png
Plot saved for V_DRONE_003.csv in /co

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

# Define paths
csv_directory = f"{OT}/FileV"
output_directory = f"{OT}/GTPlotV1"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Iterate through each file in the CSV directory
for file_name in os.listdir(csv_directory):
    file_path = os.path.join(csv_directory, file_name)

    try:
        # Read the file based on its extension
        if file_name.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_name.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Skipping unsupported file format: {file_name}")
            continue

        # Check if necessary columns exist
        if 'x_center' not in df.columns or 'y_center' not in df.columns:
            print(f"Skipping file {file_name} as it doesn't contain necessary columns")
            continue

        sorted_coords = list(zip(df['x_center'], df['y_center']))

        # Plot the center coordinates
        plt.figure(figsize=(10, 8))
        plt.scatter([coord[0] for coord in sorted_coords], [coord[1] for coord in sorted_coords], marker='o', color='green', label='Ground Truth')

        plt.xlabel('X coordinate')
        plt.ylabel('Y coordinate')
        plt.title('Center Coordinates of Bounding Boxes')
        plt.xlim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.ylim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.gca().invert_yaxis()  # YOLO format typically has y-axis inverted
        plt.legend()
        plt.grid(True)

        # Save the plot in the GT folder with the same name as the CSV/XLSX file but with a .png extension
        output_file = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.png")
        plt.savefig(output_file, transparent=True)
        plt.close()  # Close the plot to free memory

        print(f"Plot saved for {file_name} in {output_file}")
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

print("Plotting completed.")

Plot saved for V_DRONE_016.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_016.png
Plot saved for V_DRONE_014.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_014.png
Plot saved for V_DRONE_091.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_091.png
Plot saved for V_DRONE_080.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_080.png
Plot saved for V_DRONE_071.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_071.png
Plot saved for V_DRONE_067.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_067.png
Plot saved for V_DRONE_055.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_055.png
Plot saved for V_DRONE_059.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_059.png
Plot saved for V_DRONE_004.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotV1/V_DRONE_004.png
Plot saved for V_DRONE_003.c

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

# Define paths
csv_directory = f"{OT}/FileIR"
output_directory = f"{OT}/GTPlotIR"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Iterate through each file in the CSV directory
for file_name in os.listdir(csv_directory):
    file_path = os.path.join(csv_directory, file_name)

    try:
        # Read the file based on its extension
        if file_name.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_name.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Skipping unsupported file format: {file_name}")
            continue

        # Check if necessary columns exist
        if 'x_center' not in df.columns or 'y_center' not in df.columns:
            print(f"Skipping file {file_name} as it doesn't contain necessary columns")
            continue

        sorted_coords = list(zip(df['x_center'], df['y_center']))

        # Plot the center coordinates
        plt.figure(figsize=(10, 8))
        plt.scatter([coord[0] for coord in sorted_coords], [coord[1] for coord in sorted_coords], marker='o', color='green', label='Ground Truth')

        # Annotate the points with sequential numbers
        for i, (x, y) in enumerate(sorted_coords):
            plt.annotate(str(i + 1), (x, y), textcoords="offset points", xytext=(0, 10), ha='center', color='red')

        plt.xlabel('X coordinate')
        plt.ylabel('Y coordinate')
        plt.title('Center Coordinates of Bounding Boxes')
        plt.xlim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.ylim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.gca().invert_yaxis()  # YOLO format typically has y-axis inverted
        plt.legend()
        plt.grid(True)

        # Save the plot in the GT folder with the same name as the CSV/XLSX file but with a .png extension
        output_file = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.png")
        plt.savefig(output_file, transparent=True)
        plt.close()  # Close the plot to free memory

        print(f"Plot saved for {file_name} in {output_file}")
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

print("Plotting completed.")

Plot saved for IR_DRONE_026.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_026.png
Plot saved for IR_DRONE_011.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_011.png
Plot saved for IR_DRONE_002.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_002.png
Plot saved for IR_DRONE_003.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_003.png
Plot saved for IR_DRONE_036.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_036.png
Plot saved for IR_DRONE_023.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_023.png
Plot saved for IR_DRONE_040.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_040.png
Plot saved for IR_DRONE_013.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_013.png
Plot saved for IR_DRONE_012.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR/IR_DRONE_012.png
Plot saved

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

# Define paths
csv_directory = f"{OT}/FileIR"
output_directory = f"{OT}/GTPlotIR1"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Iterate through each file in the CSV directory
for file_name in os.listdir(csv_directory):
    file_path = os.path.join(csv_directory, file_name)

    try:
        # Read the file based on its extension
        if file_name.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_name.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Skipping unsupported file format: {file_name}")
            continue

        # Check if necessary columns exist
        if 'x_center' not in df.columns or 'y_center' not in df.columns:
            print(f"Skipping file {file_name} as it doesn't contain necessary columns")
            continue

        sorted_coords = list(zip(df['x_center'], df['y_center']))

        # Plot the center coordinates
        plt.figure(figsize=(10, 8))
        plt.scatter([coord[0] for coord in sorted_coords], [coord[1] for coord in sorted_coords], marker='o', color='green', label='Ground Truth')

        plt.xlabel('X coordinate')
        plt.ylabel('Y coordinate')
        plt.title('Center Coordinates of Bounding Boxes')
        plt.xlim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.ylim(0, 1)  # Assuming coordinates are normalized (0 to 1)
        plt.gca().invert_yaxis()  # YOLO format typically has y-axis inverted
        plt.legend()
        plt.grid(True)

        # Save the plot in the GT folder with the same name as the CSV/XLSX file but with a .png extension
        output_file = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.png")
        plt.savefig(output_file, transparent=True)
        plt.close()  # Close the plot to free memory

        print(f"Plot saved for {file_name} in {output_file}")
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

print("Plotting completed.")

Plot saved for IR_DRONE_026.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_026.png
Plot saved for IR_DRONE_011.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_011.png
Plot saved for IR_DRONE_002.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_002.png
Plot saved for IR_DRONE_003.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_003.png
Plot saved for IR_DRONE_036.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_036.png
Plot saved for IR_DRONE_023.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_023.png
Plot saved for IR_DRONE_040.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_040.png
Plot saved for IR_DRONE_013.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_013.png
Plot saved for IR_DRONE_012.csv in /content/gdrive/MyDrive/FYP_Drone/ObjectTracking/GTPlotIR1/IR_DRONE_012.png
P